Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return tf.compat.as_str(f.read(name))
  f.close()
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [6]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
    print(self._cursor)
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, 3, num_unrollings)
valid_batches = BatchGenerator(valid_text, 3, 2)
# b = train_batches.next()
# s = [''] * 3
# s = [''.join(x) for x in zip(s, ['q','v','e'])]
# s = [''.join(x) for x in zip(s, ['q','v','e'])]
# print(b[0], s)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

[1, 33333001, 66666001]
[1, 334, 667]
['ons anarchi', 't via elect', 'everal opti']
['ists advoca', 'tromagnetic', 'ions for br']
[' an', ' ta', ' so']
['nar', 'ake', 'oci']
[1, 1562485, 3124969, 4687453, 6249937, 7812421, 9374905, 10937389, 12499873, 14062357, 15624841, 17187325, 18749809, 20312293, 21874777, 23437261, 24999745, 26562229, 28124713, 29687197, 31249681, 32812165, 34374649, 35937133, 37499617, 39062101, 40624585, 42187069, 43749553, 45312037, 46874521, 48437005, 49999489, 51561973, 53124457, 54686941, 56249425, 57811909, 59374393, 60936877, 62499361, 64061845, 65624329, 67186813, 68749297, 70311781, 71874265, 73436749, 74999233, 76561717, 78124201, 79686685, 81249169, 82811653, 84374137, 85936621, 87499105, 89061589, 90624073, 92186557, 93749041, 95311525, 96874009, 98436493]
[1]
['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi'

In [7]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [10]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [56]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.295822 learning rate: 10.000000
Minibatch perplexity: 27.00
u  aifiwe lct zloteebefou eemhgtiulo inxdop  bocnt srhe  o orav inixolqye rknzjc
stafza o swzspebeebs ysogoywdrreerwmkvumdcpxh  raaeekarngtsgtgng ezdjnvajh e y m
orvkh hnn aejtma ftij pvh jmus c mhkudidbqtnwqsea oeaeddjxkglq zazonro chg idnef
utzdnwye ror  krzct hefbdefdlnmga    jho xxi   k sboofrudsvapmni acyy o i r sisk
boga ikweuk sgineem trhpz rrft rez inln ge lmxf dajn et  dne ediroflkfa dsjmtwoc
Validation set perplexity: 19.57
Average loss at step 100: 2.853061 learning rate: 10.000000
Minibatch perplexity: 20.26
Validation set perplexity: 15.24
Average loss at step 200: 2.682040 learning rate: 10.000000
Minibatch perplexity: 16.66
Validation set perplexity: 11.78
Average loss at step 300: 2.515027 learning rate: 10.000000
Minibatch perplexity: 12.63
Validation set perplexity: 11.86
Average loss at step 400: 2.438634 learning rate: 10.000000
Minibatch perplexity: 10.96
Validation se

KeyboardInterrupt: 

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [143]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
    xBig = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes * 4], -0.1, 0.1))
    mBig = tf.Variable(tf.truncated_normal([num_nodes, num_nodes * 4], -0.1, 0.1))
    bBig = tf.Variable(tf.zeros([1, num_nodes * 4]))
    
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
    
                                            
    def lstm_cell(i, o, state):
        gateBig = tf.matmul(i, xBig) + tf.matmul(o, mBig) + bBig
        
        inGate = tf.sigmoid(gateBig[:,0:num_nodes])
        forgetGate = tf.sigmoid(gateBig[:, num_nodes:2*num_nodes])
        outGate = tf.sigmoid(gateBig[:, 2*num_nodes:3*num_nodes])
        update = tf.sigmoid(gateBig[:, 3*num_nodes:4*num_nodes])
        state = forgetGate * state + inGate * tf.tanh(update)
        
        return outGate * tf.tanh(state), state
    
    # Input data.
    train_data = list()
    for i in range(num_unrollings + 1):
        train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
    train_inputs = train_data[0:num_unrollings]
    train_labels = train_data[1:num_unrollings+1]
    
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)
    
    with tf.control_dependencies([saved_output.assign(output),saved_state.assign(state)]):
        logits = tf.matmul(tf.concat(0, outputs), w) + b
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf.concat(0, train_labels)))
        
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
    
    # instead of tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    # preprocess gradients
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    # now processing futher gradient apply
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)
    
    train_prediction = tf.nn.softmax(logits)
    
    sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes])))
    
    sample_output, sample_state = lstm_cell(sample_input, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                  saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.matmul(sample_output, w) + b)
    
    

In [62]:
num_steps = 1001
summary_frequency = 100

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    
    for step in range(num_steps):
        
        batches = train_batches.next()
        feed_data = dict()
        for i in range(num_unrollings+1):
            feed_data[train_data[i]] = batches[i]
            
        _, l, predictions, lr = session.run(
            [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        
        if(step % 100 == 0):
            print(step)
    
    
    b = valid_batches.next()
    predictions = sample_prediction.eval({sample_input: b[0]})
    valid_logprob = logprob(predictions, b[1])
    print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob)))
    
    print('=' * 80)
    for _ in range(5):
        feed = sample(random_distribution())
        sentence = characters(feed)[0]
        reset_sample_state.run()
        for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
        print(sentence)
    print('=' * 80)

Initialized
0
100
200
300
400
500
600
700
800
900
1000
Validation set perplexity: 83.83
karates apal tu red yane invan boron one nics ngtweentes of cent in bo opnasties
uarader poxe sely comsets opivisibn dishieldetwie ne ingan boudes not zorce zo p
musicover opent deve the of borce sevsties yates lost seduce efnt icsaand tyibha
ntts the matarated by sevs ond cent of control ofer ont the seasedr prverelly yi
zerel ppodea behavere poane boudes in groxer one nigne in nedicen born bo ned on


---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

In [24]:

def encode_bigram(chars):
    #print(chars)
    c1 = char2id(chars[0])
    c2 = char2id(chars[1])
    return c1* vocabulary_size + c2

def dencode_bigram(num):
    id2 = num % vocabulary_size
    id1 = (num-id2)//vocabulary_size
    return id2char(id1) + id2char(id2)

class BigramBatchGenerator:
    
    def __init__(self, text, size, unrolls): # "abcdefghijklmn", 4, 1
        self._text = text # "abcdefghijklmn"
        self._len = len(text) # 14
        self._batch_size = size # 4
        self._unrolls = unrolls # 1
        segments = self._len // (size) # 12 / 4 = 3
        self._cursors = [offset * segments for offset in range(size)] # [0,3,6,9]
        self._last_seg = self._next_batch() # --> ['ab', 'de', 'gh', 'jk']
        #print(segments, self._len, self._cursors, [text[i] for i in self._cursors])
        
    def _next_batch(self):
        batch = np.zeros(shape=(self._batch_size), dtype=np.int) # [0,0,0,0]
        for b in range(self._batch_size):
            cur = self._cursors[b]
            batch[b] = encode_bigram(self._text[cur:cur+2])
            self._cursors[b] = (cur + 2) % self._len
        return batch
    
    def next(self):
        batch = [self._last_seg]
        for i in range(self._unrolls):
            batch.append(self._next_batch())
        self._last_seg = batch[-1]
        return batch
            
b = BigramBatchGenerator("abcdefghijklmn", 4, 1).next()
for _b in b:
    print([dencode_bigram(i) for i in _b])
    
train_bibatches = BigramBatchGenerator(train_text, batch_size, num_unrollings)
valid_bibatches = BigramBatchGenerator(valid_text, 1, 2)


['ab', 'de', 'gh', 'jk']
['cd', 'fg', 'ij', 'lm']


In [19]:
def biRandOut():
    return random.randint(0, vocabulary_size * vocabulary_size - 1)

In [33]:
embedding_size = 10
node_count = 64
num_nodes = 64


graph = tf.Graph()
with graph.as_default():
    
        gate_count = 4
        # Parameters:
        embeddings = tf.Variable(tf.random_uniform([vocabulary_size * vocabulary_size, embedding_size], -1.0, 1.0))
        
        # Gates: input, previous output, and bias.
        input_weights = tf.Variable(tf.truncated_normal([embedding_size, node_count * gate_count], -0.1, 0.1))
        output_weights = tf.Variable(tf.truncated_normal([node_count, node_count * gate_count], -0.1, 0.1))
        bias = tf.Variable(tf.zeros([1, node_count * gate_count]))
        # Variables saving state across unrollings.
        saved_output = tf.Variable(tf.zeros([batch_size, node_count]), trainable=False)
        saved_state = tf.Variable(tf.zeros([batch_size, node_count]), trainable=False)
        # Classifier weights and biases.
        w = tf.Variable(tf.truncated_normal([node_count, embedding_size], -0.1, 0.1))
        b = tf.Variable(tf.zeros([embedding_size]))

        # Definition of the cell computation.
        def lstm_cell(i, o, state):
            """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
            Note that in this formulation, we omit the various connections between the
            previous state and the gates."""
            values = tf.split(1, gate_count, tf.matmul(i, input_weights) + tf.matmul(o, output_weights) + bias)
            input_gate = tf.sigmoid(values[0])
            forget_gate = tf.sigmoid(values[1])
            update = values[2]
            state = forget_gate * state + input_gate * tf.tanh(update)
            output_gate = tf.sigmoid(values[3])
            return output_gate * tf.tanh(state), state

        # Input data.
        train_data = list()
        train_embeddings = list()
        for i in range(num_unrollings + 1):
            train_data.append(tf.placeholder(tf.int32, shape=[batch_size], name="train"+str(i)))
            train_embeddings.append(tf.nn.embedding_lookup(embeddings, train_data[-1]))
        train_inputs = train_embeddings[:num_unrollings]
        train_labels = train_embeddings[1:]  # labels are inputs shifted by one time step.

        # Unrolled LSTM loop.
        outputs = list()
        output = saved_output
        state = saved_state
        for i in train_inputs:
            output, state = lstm_cell(i, output, state)
            outputs.append(output)

        # State saving across unrollings.
        with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
            # Classifier.
            logits = tf.matmul(tf.concat(0, outputs), w)+ b
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf.concat(0, train_labels)))

        # Optimizer.
        global_step = tf.Variable(0)
        learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        gradients, v = zip(*optimizer.compute_gradients(loss))
        gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
        optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

        # Predictions.
        train_prediction = tf.nn.softmax(logits)

        embedding_norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
        embedding_normalized = embeddings / embedding_norm

        # Sampling and validation eval: batch 1, no unrolling.
        sample_input = tf.placeholder(tf.int32, shape=[1])
        saved_sample_output = tf.Variable(tf.zeros([1, node_count]))
        saved_sample_state = tf.Variable(tf.zeros([1, node_count]))
        reset_sample_state = tf.group(
            saved_sample_output.assign(tf.zeros([1, node_count])),
            saved_sample_state.assign(tf.zeros([1, node_count]))
        )
        sample_embedding = tf.nn.embedding_lookup(embeddings, sample_input)
        sample_output, sample_state = lstm_cell(sample_embedding, saved_sample_output, saved_sample_state)
        with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                      saved_sample_state.assign(sample_state)]):
            sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

        
    
    

In [34]:
num_steps = 11
summary_frequency = 100

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    
    for step in range(num_steps):
        
        batches = train_bibatches.next()
        feed_dict = dict()
        for i in range(num_unrollings+1):
            feed_dict[train_data[i]] = batches[i]
            
        _, l, predictions, lr = session.run(
            [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        
        if(step % 100 == 0):
            print(step)
    
    
    b = valid_bibatches.next()
    print(b)
    predictions = sample_prediction.eval({sample_input: b[0]})
    print(predictions)
    valid_logprob = logprob(predictions, b[1])
    print(valid_logprob)
    print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob)))
    
#     print('=' * 80)
#     for _ in range(5):
#         feed = biRandOut()
#         print(feed)
#         sentence = characters(feed)[0]
#         print(sentence)
#         reset_sample_state.run()
#         for _ in range(79):
#             prediction = sample_prediction.eval({sample_input: feed})
#             print(prediction)
#             feed = biRandOut()
#             sentence += characters(feed)[0]
#         print(sentence)
#     print('=' * 80)

Initialized
0
[array([1]), array([20]), array([153])]
[[  3.73417526e-01   2.74925151e-05   8.54790136e-02   3.67987996e-09
    4.67816055e-01   5.40416245e-08   6.66283420e-04   5.76165871e-09
    3.26472644e-07   7.25932643e-02]]
1894.06707048
Validation set perplexity: inf


---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---